In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from src.config.models import DatabaseConfig, LLMConfig, PromptConfig

db_config = DatabaseConfig(
    type="postgres",
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password=os.getenv("POSTGRES_PWD"),
    schema="video_games"
)

In [2]:
from src.connettori.postgres import PostgresManager

db = PostgresManager(
    host=db_config.host,
    port=db_config.port,
    database=db_config.database,
    user=db_config.user,
    password=db_config.password
)
db.connect()

True

In [3]:
from src.dbcontext.postgres_metadata_retriever import PostgresMetadataRetriever

metadata_retriever = PostgresMetadataRetriever(db.engine, schema=db_config.schema)

print(metadata_retriever.get_table_definition("genre"))

CREATE TABLE genre (id integer NOT NULL, genre_name character varying(50));


In [4]:
from src.llm_handler.ollama_handler import OllamaHandler

llm = OllamaHandler(
    base_url="http://localhost:11434",
    model="llama3.1"
)

In [5]:
from src.llm_input.prompt_generator import PromptGenerator

prompt_config = PromptConfig(include_sample_data=True, max_sample_rows=3)
prompt_generator = PromptGenerator(
    metadata_retriever=metadata_retriever,
    schema_name=db_config.schema,
    prompt_config=prompt_config
)

In [6]:
user_question = "Mostrami i primi 5 giochi del database"
prompt = prompt_generator.generate_prompt(user_question)
response = llm.get_completion(prompt)

Errore nella richiesta API: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F6A6D8B1C0>: Failed to establish a new connection: [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione'))


In [ ]:
from src.llm_output.response_handler import ResponseHandler
response_handler = ResponseHandler(db, db_config.schema)
results = response_handler.process_response(response)

if results["success"]:
    print("Query SQL:", results["query"])
    print("\nSpiegazione:", results["explanation"])
    print("\nRisultati:", results["results"])